<nav class="navbar navbar-default">
  <div class="container-fluid">
    <div class="navbar-header">
      <a class="navbar-brand" href="_Dataoving2.ipynb">Dataøving 2</a>
    </div>
    <ul class="nav navbar-nav">
        <li><a href="Oppgave1.ipynb">Oppgave1 - Digitale Signal og Diskrét Datarepresentasjon.</a></li>
        <li><a href="Oppgave2.ipynb">Oppgave 2 - Signalanalyse med DFT.</a></li>
        <li class="active"><a href="Oppgave3.ipynb">Oppgave 3 - Signalgenerasjon med IDFT.</a></li>
        <li><a href="Oppgave4.ipynb">Oppgave 4 - Reparasjon av lydsignal.</a></li>
    </ul>
  </div>
</nav>

# Signalgenerasjon med IDFT

__Læringsmål__:
* Sammenhengen mellom en DFT og komposisjon av et signal.
* Hvordan legge til, fjerne og justere på sinuskomponenter i et signal med bruk av IDFT.

__Bibliotek og notebook-konfigurasjon:__

In [2]:
from numpy import sin, cos, pi, exp, mean, absolute, angle, conj # Sentrale matematiske funksjoner
from numpy.fft import fft, ifft             # DFT og IDFT
import numpy as np                          # Importer funksjonalitet fra numpy biblioteket med prefiks "np"
import matplotlib.pyplot as plt             # Importer pyplot modulen i matplotlib med prefiks "plt"
from funksjoner_data3 import timeSpectrumStem

%matplotlib ipympl

**Introduksjon**

Fouriertransformasjon kan brukes til mer enn å identifisere frekvenskomponenter i et signal. En DFT er en *fullstendig* dekomposisjon av et signal, som betyr at den inneholder all informasjon som er nødvendig for å rekonstruere et signal fra frekvenskomponentene. 

I denne oppgaven skal vi konstruere signal ved å legge til og modifisere komponentene i frekvensplanet, for så å regne oss frem til et signal i tidsdomenet med bruk av IDFT. Nedenfor føgler et eksempel som viser matematikken rundt signalgenerasjon med IDFT. Signalet som genereres er 64 sampler av en sinussekvens gitt ved formelen $x[n] = 2\cdot \cos\left(2\pi \frac{4}{64}\cdot n - \frac{\pi}{3} \right), \ \ \ 0 \leq n < 64$.
* Finn ut hvilkde indekser i DFT-sekvensen som vil korrespondere med et sinussignal med normalisert vinkelfrekvens $\hat{\omega} = 2\pi \frac{4}{64}$:
$$\pm \hat{\omega}_k = \pm 2\pi \cdot \frac{4}{64} = \pm 2\pi \cdot \frac{k}{N}, \ \ \ N = 64$$
$$k = \pm 4 $$ 
$$ \downarrow$$
$$\text{(negative tall 'ruller over' og teller ned fra høyeste indeks.) }$$
$$ \downarrow$$
$$ k = \pm 4 \% 64$$
 
$$
k =
\begin{cases}
\begin{align}
4\ \ \ & \text{(positiv frekvenskomponent)} \\
-4 \% 64 = 64 - 4 = 60\ \ \ & \text{(negativ frekvenskomponent)}
\end{align}
\end{cases}
$$
 

* Finn den komplekse amplituden $a_k$ slik at signalet kan skrives som $x[n] = a_k \cdot e^{j\hat{\omega}_k\cdot n} + a_k^{*} \cdot e^{-j\hat{\omega}_k\cdot n}$:
$$2\cdot \cos \left(2\pi \frac{4}{64}\cdot n - \frac{\pi}{3} \right) = e^{-j\frac{\pi}{3}}\cdot e^{j2\pi \frac{4}{64}\cdot n} + e^{j\frac{\pi}{3}}\cdot e^{-j2\pi \frac{4}{64}\cdot n}$$
$$a_4 = e^{-j\frac{\pi}{3}}$$
 
 
* Sett de aktuelle elementene i $X[k]$ til riktig verdi *(her må vi multiplisere med antall sampler i DFT-sekvensen)*
$$
X[k] = 
\begin{cases}
\begin{align}
N \cdot a_4= 64 \cdot e^{-j\frac{\pi}{3}},\ \ \  & k = 4 \\
N \cdot a_4^{*} = 64\cdot e^{j\frac{\pi}{3}},\ \ \  & k = 60 \\
0 \ \ \ & ellers
\end{align}
\end{cases}
$$
 
 
* Invers DFT gir det ønskede signalet:
$$x[n] = IDFT(X[k])$$


Kodecellen nedenfor utfører de to siste stegene i denne prosessen. Parmaetrene `k`, `A` og `phi` er arrays, noe som lar oss bygge et signal med én eller flere sinussekvenser. Foreløpig vil koden kun generere signalet fra utregningen ovenfor. 

*NB! legg merke til at vi kun trenger å angi de positive frekvenskomponentene. Kode for å regne ut det komplekskonjugerte "speilbildet" som inneholder de negative frekvenskomponentene er allerede inkludert.*

In [13]:
# Parametre for sinusledd(ene)
N = 96                         # Antall sampler DFT'en skal representere
k = np.array([6, 9])              # DFT-indeks(er) = antall svingninger i løpet av N sampler
A = np.array([2, 2.3])              # Amplitude(r)
phi = np.array([-pi/3, 3*pi/5])        # Fase(r)
phi += -17*2*pi*k/N               # Oppgave c

# Utregning av signalsampler
Xk = 1j*np.zeros(N)             # Lag et "tomt" komplekst signal med lengde N
a_k = A*exp(1j*phi)/2.0         # Kompleks amplitude (A*e^(j*phi)) positiv frekvenskomponent 
Xk[k] = a_k*N                   # Sett DFT-verdier for de positive frekvenskomponentene
Xk[N-k] = conj(a_k)*N           # Sett DFT-verdier for de negative frekvenskomponentene

xn = ifft(Xk)

plt.close(1); plt.figure(1)
if(np.sum(np.absolute(np.imag(xn)))>1e-10):
    print("DFT conjugate symmetry not maintained. IDFT produced complex signal.")
else:
    timeSpectrumStem(np.real(xn), Xk, N)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Kjør gjennom koden og studer figurene som blir generert. Det kan også være en god idé å justere på noen av parametrene og observere hva resultatet blir.

## a) 
* Legg til en ny sinuskomponent med normalisert vinkelfrekvens $\hat{\omega} = \frac{3\pi}{16}$, amplitude $A = 2.3$ og fase $\phi = \frac{3\pi}{5}$ ved å legge til et nytt element arrayene `k`, `A` og `phi`. 

## b) 
* Vi vil generere flere sampler av samme signal. Gjør ytterligere endringer på variablene `k`, `A` og `phi` i tillegg til `N` for å produsere 96 sampler istedenfor 64. Pass på at vinkelfrekvensen til sinuskomponentene forblir uendret. 

## c)

* Påfør hele signalet $M=-17$ sampler tidsforskyvning ved å justere på fasen til frekvenskomponentene.